# Data processing

In [1]:
# Import libraries
import pandas as pd
import os
import json
import re
import numpy as np
import nltk
# from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import string
pd.options.mode.chained_assignment = None


# nie usuwam interpunkcji dla lyrics, robie to dopiero dla korpusu
# https://likegeeks.com/python-remove-punctuation/#Remove_all_punctuation_except_apostrophe

In [2]:
# Read csv data with songs to process
# pd.set_option("display.max_rows", None, "display.max_columns", None)
songs = pd.read_csv('all_time_billboard_wrap_up_cleaned.csv')
display(songs)

,Position,Artist,Song Title,Year,JSON file,Split Names,Song Language
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,0.json,Gordon Jenkins,en
1,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,en
2,4,Gary and Bing Crosby,Sam's Song,1950,3.json,Gary,en
3,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,en
4,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,en
...,...,...,...,...,...,...,...
6405,95,Cole Swindell,Single Saturday Night,2021,6774.json,Cole Swindell,en
6406,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,en
6407,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,en
6408,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,en


### Tokenizing and lemmatizing

In [3]:
# Download Punkt sentence tokenizing, parts of speech and lemmatizing models
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [4]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)

# Create columns with lyrics, tokenized lyrics, particular parts of speech and lemmatized words
wd = os.getcwd()
jsons_path = wd + '\\jsons\\'
stop = stopwords.words('english')
stop_list = ['im', 'dont', 'lyrics', 'verse', 'intro', 'lyric', 'aint', 'ill', 'ive', 't', 's', 'm', 'll', 're', 'd', 'don', 've', 'gon', 'don', 'wheres', 'whats', 'hows']
stop.extend(stop_list)
lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()
my_punctuation = '!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~' # pass apos
songs['Lyrics'] = ''
songs['Corpus'] = [[] for _ in range(len(songs))]
songs['Tokenized Lyrics']  = [[] for _ in range(len(songs))]
songs['Lemmatized Words'] = [[] for _ in range(len(songs))]
songs['Parts of Speech'] = [[] for _ in range(len(songs))]
songs['Adjectives'] = [[] for _ in range(len(songs))] # JJ
songs['Nouns'] = [[] for _ in range(len(songs))] # NN
songs['Adverbs'] = [[] for _ in range(len(songs))] # RB
songs['Pronouns'] = [[] for _ in range(len(songs))] # PRB
songs['Verbs'] = [[] for _ in range(len(songs))] # VB
for i in range(len(songs)):
    with open(jsons_path + songs['JSON file'][i], 'r', encoding = 'UTF-8') as f:
        data = json.load(f)['lyrics'].lower()
        # Delete additional blank lines and comments in square brackets like [Verse 1], [Intro]
        # https://stackoverflow.com/questions/14596884/remove-text-between-and
        # https://stackoverflow.com/questions/1140958/whats-a-quick-one-liner-to-remove-empty-lines-from-a-python-string
        lyrics = re.sub(r'[\[].*?[\]]', ' ', data)
        lyrics = os.linesep.join([j for j in lyrics.splitlines() if j])
        lyrics = lyrics.split('lyrics')[1].strip() # removing initial pattern from json: "[song title] lyrics"
    if type(lyrics) == 'float': # if lyrics is NaN
        lyrics = ''
    songs['Lyrics'][i] = ' '.join(lyrics.split('\r\n'))
    corpus = songs['Lyrics'][i].translate(str.maketrans('', '', my_punctuation))
    corpus = corpus.split()
    songs['Corpus'][i] = ' '.join([word for word in corpus if word not in stop])
    tokenized_lyrics = tokenizer.tokenize(songs['Corpus'][i])
    songs['Tokenized Lyrics'][i] = tokenized_lyrics
    songs['Lemmatized Words'][i] = [lemmatizer.lemmatize(word, 'v') for word in tokenized_lyrics]
    songs['Parts of Speech'][i] = nltk.pos_tag(songs['Lemmatized Words'][i])
    songs['Adjectives'][i] = []
    songs['Nouns'][i] = []
    songs['Adverbs'][i] = []
    songs['Pronouns'][i] = []
    songs['Verbs'][i] = []
    for m in songs['Parts of Speech'][i]:
        if m[1] == 'JJ':
            songs['Adjectives'][i].append(m[0])
        elif m[1] == 'NN':
            songs['Nouns'][i].append(m[0])
        elif m[1] == 'RB':
            songs['Adverbs'][i].append(m[0])
        elif m[1] == 'PRP':
            songs['Pronouns'][i].append(m[0])
        elif m[1] == 'VB':
            songs['Verbs'][i].append(m[0])
            
songs = songs[songs['Lyrics'] != '']
            
display(songs)

,Position,Artist,Song Title,Year,JSON file,Split Names,Song Language,Lyrics,Corpus,Tokenized Lyrics,Lemmatized Words,Parts of Speech,Adjectives,Nouns,Adverbs,Pronouns,Verbs
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,0.json,Gordon Jenkins,en,irene goodnight irene goodnight goodnight iren...,irene goodnight irene goodnight goodnight iren...,"[irene, goodnight, irene, goodnight, goodnight...","[irene, goodnight, irene, goodnight, goodnight...","[(irene, NN), (goodnight, VBD), (irene, JJ), (...","[irene, goodnight, irene, i'll, last, saturday...","[irene, goodnight, irene, see, dream, night, w...","[sometimes, sometimes, sometimes, fireside]",[],"[get, settle, take, take, stop, stay]"
1,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,en,"mona lisa, mona lisa, men have named you you'r...",mona lisa mona lisa men named like lady mystic...,"[mona, lisa, mona, lisa, men, named, like, lad...","[mona, lisa, mona, lisa, men, name, like, lady...","[(mona, NN), (lisa, NN), (mona, NN), (lisa, VB...","[lady, mystic, blame, mona, lisa, hide, break,...","[mona, lisa, mona, name, smile, cause, lisa, s...","[lonely, lonely, art, lonely, art]",[],[]
2,4,Gary and Bing Crosby,Sam's Song,1950,3.json,Gary,en,"ah, here's a happy tune, you'll love to croon ...",ah here's happy tune love croon call sam's son...,"[ah, here's, happy, tune, love, croon, call, s...","[ah, here's, happy, tune, love, croon, call, s...","[(ah, NN), (here's, NN), (happy, JJ), (tune, N...","[happy, hum, grim, street, whistle, agree, gra...","[ah, here's, tune, croon, call, sam's, song, c...","[soon, never, learn, really, sly, still, agree...",[],"[love, find, pay, keep, tell, find]"
3,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,en,won't you play some simple melody like my moth...,play simple melody like mother sang one good o...,"[play, simple, melody, like, mother, sang, one...","[play, simple, melody, like, mother, sing, one...","[(play, NN), (simple, JJ), (melody, NN), (like...","[simple, good, old, simple, musical, classical...","[play, melody, mother, fashion, harmony, play,...","[simply, simply, simply]",[],[play]
4,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,en,put another nickel in in the nickelodeon all i...,put another nickel nickelodeon want music musi...,"[put, another, nickel, nickelodeon, want, musi...","[put, another, nickel, nickelodeon, want, musi...","[(put, VB), (another, DT), (nickel, NN), (nick...","[want, kiss, dear, danciong, want, kiss, dear,...","[nickel, nickelodeon, music, music, music, i'd...","[close, close]",[],"[put, keep]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405,95,Cole Swindell,Single Saturday Night,2021,6774.json,Cole Swindell,en,i was out taking shots throwing down at the sp...,taking shots throwing spot three sheets guys e...,"[taking, shots, throwing, spot, three, sheets,...","[take, shots, throw, spot, three, sheet, guy, ...","[(take, VB), (shots, NNS), (throw, VB), (spot,...","[single, light, single, right, sittin, ’, red,...","[spot, sheet, guy, saturday, night, time, crus...","[right, nowhere, nowhere, pretty, long, even, ...",[us],"[take, throw, kill, try, catch, know, find, sh..."
6406,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,en,i can hook a trailer on a two-inch hitch i can...,hook trailer twoinch hitch shoot shotgun catch...,"[hook, trailer, twoinch, hitch, shoot, shotgun...","[hook, trailer, twoinch, hitch, shoot, shotgun...","[(hook, NN), (trailer, NN), (twoinch, NN), (hi...","[fish, stay, tough, good, late, give, wrong, k...","[hook, trailer, twoinch, hitch, shoot, shotgun...","[fore, boy, really, yeah, fore, boy, really, y...",[],"[know, love, love, let, go, take, get, know, l..."
6407,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,en,"(what's happenin', chi chi?) sexy lil' bitch...",what's happenin' chi chi sexy lil' bitch sexy ...,"[what's, happenin, ', chi, chi, sexy, lil, ', ...","[what's, happenin, ', c

In [5]:
songs.to_csv(wd + '\\lyrics_processed_data.csv', index = False)